In [1]:
from pathlib import Path

thispath = Path('__file__').resolve()
base_path = thispath.parent.parent
import sys; sys.path.insert(0, str(base_path))
from dataset.copd_dataset import DirLabCOPD

In [2]:

NORMALIZATION_CFG = {
    'norm_type': 'min-max',
    'mask': None,
    'max_val': 255,
    'window': [-1024, 600],
    'dtype': 'uint8',
}

data = DirLabCOPD(
    data_path=base_path/'data',
    cases=['all'],
    partitions=['train'],
    return_lm_mask=True,
    normalization_cfg=None,
    return_body_masks=True,
    return_lung_masks=True
)

In [3]:
import numpy as np

def get_landmarks_from_array(lm_mask: np.ndarray) -> np.ndarray:
    """_summary_

    Args:
        lm_mask (np.ndarray): mask containing landmark points where each point
            has a label coinciding to its index in the original dataset .txt.
    Returns:
        np.ndarray: array containing the x,y,z coordinates of each of the landmark
            points orden according to the label index it has.
    """
    locs = np.zeros((300, 3))
    for i in np.unique(lm_mask):
        if i != 0:
            i = int(i)
            x, y, z = np.where(lm_mask == i)
            locs[i, :] = np.array([x[0], y[0], z[0]])
            
    return locs


In [7]:
from utils.metrics import target_registration_error
import elastix.elastix_utils as e_utils
from utils.utils import save_img_from_array_using_metadata, get_landmarks_array_from_txt_file
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm
import time

# Define parameter maps to use
# param_maps_to_use = ['Par0003.affine.txt']

# param_maps_to_use = [
    # 'Parameters.MI.Coarse.Bspline_tuned.txt',
    # 'Parameters.MI.Fine.Bspline_tuned.txt',
    # 'Parameters.MI.RP.Bspline_tuned.txt'
# ]

param_maps_to_use = [
    'Parameters.Par0008.affine.txt',
    'Parameters.Par0008.elastic.txt'
]

# param_maps_to_use = [
    # 'Par0003.affine.txt',
    # 'Par0003.bs-R1-fg.txt',
    # 'Par0003.bs-R2-fg.txt',
    # 'Par0003.bs-R3-fg.txt',
    # 'Par0003.bs-R4-fg.txt',
    # 'Par0003.bs-R5-fg.txt',
    # 'Par0003.bs-R6-fg.txt',
    # 'Par0003.bs-R7-fg.txt',
    # 'Par0003.bs-R8-fg.txt'
# ]
# param_maps_to_use = [
#     'Par0003.affine.txt',
#     'Par0003.bs-R1-ug.txt',
#     'Par0003.bs-R2-ug.txt',
#     'Par0003.bs-R3-ug.txt',
#     'Par0003.bs-R4-ug.txt',
#     'Par0003.bs-R5-ug.txt',
#     'Par0003.bs-R6-ug.txt',
#     'Par0003.bs-R7-ug.txt',
#     'Par0003.bs-R8-ug.txt'
# ]

output_path = Path('/home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/experiments/elastix')
# params_path = Path('/home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/elastix/parameter_maps/Par0003')
params_path = Path('/home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/elastix/parameter_maps/Par0008')
tres = []
# Define fixed imgage to use
for i in tqdm(range(len(data))):
    sample = data[i]
    # for pm in param_maps_to_use:
    # Define output paths
    pm = param_maps_to_use[-1]
    output_pm_path = output_path / pm.rstrip('.txt')
    output_pm_path.mkdir(exist_ok=True, parents=True)

    # Read and modify parameters file
    parameters_filename = params_path / pm
    result_path = output_pm_path / sample['case']
    result_path.mkdir(exist_ok=True, parents=True)
    
    # field_value_pairs = [('WriteResultImage', 'true'), ('ResultImageFormat', 'nii.gz')]
    field_value_pairs = [
        ('WriteResultImage', 'true'),
        ('ResultImageFormat', 'nii.gz'),
        # ('Metric', ['AdvancedMattesMutualInformation', 'TransformRigidityPenalty'])
        # MattesMutualInformationWithRigidityPenalty
    ]
    e_utils.modify_field_parameter_map(field_value_pairs, parameters_filename)

    # Create temporary image files with the preprocessings included and 
    # also for the selected binary masks.
    res_path = result_path / 'res_tmp'
    res_path.mkdir(exist_ok=True, parents=True)

    param_maps_to_use_ = [str(params_path / p) for p in param_maps_to_use] 
    # Inhale
    i_temp_path = res_path / 'i_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['i_img'], [0,1,2], [2,1,0]), sample['ref_metadata'], i_temp_path)
    i_body_mask_temp_path = res_path / 'i_body_mask_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['i_body_mask'], [0,1,2], [2,1,0]), sample['ref_metadata'], i_body_mask_temp_path)
    i_lungs_mask_temp_path = res_path / 'i_lungs_mask_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['i_lung_mask'], [0,1,2], [2,1,0]), sample['ref_metadata'], i_lungs_mask_temp_path)

    # Exhales
    e_temp_path = res_path / 'e_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['e_img'], [0,1,2], [2,1,0]), sample['ref_metadata'], e_temp_path)
    e_body_mask_temp_path = res_path / 'e_body_mask_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['e_body_mask'], [0,1,2], [2,1,0]), sample['ref_metadata'], e_body_mask_temp_path)
    e_lungs_mask_temp_path = res_path / 'e_lungs_mask_img.nii.gz'
    save_img_from_array_using_metadata(
        np.moveaxis(sample['e_lung_mask'], [0,1,2], [2,1,0]), sample['ref_metadata'], e_lungs_mask_temp_path)

    # Register
    print('Estimating transformation...')
    start = time.time()
    transform_map_path = e_utils.elastix_wrapper(
        e_temp_path, i_temp_path, res_path.parent, param_maps_to_use_,
        e_body_mask_temp_path, i_body_mask_temp_path, verbose=False, keep_just_useful_files=False
    )
    print(f'Reg time: {time.time()-start}')
    case_path = Path(sample['i_img_path']).parent
    
    name = f"{sample['case']}_300_iBH_xyz_r1.txt"
    lm_points_filepath = case_path / name

    # Correct transformation parameters file
    field_value_pairs = [
        ('ResultImageFormat', 'nii.gz'),
        ('ResultImagePixelType', "float"),
        # ('FinalBSplineInterpolationorder', '0')
    ]
    e_utils.modify_field_parameter_map(field_value_pairs, transform_map_path)

    # Transform image
    print('Transforming points...')
    lm_out_filepath = res_path.parent / f'r_{name}'
    e_utils.transformix_wrapper(
        lm_points_filepath, lm_out_filepath, transform_map_path,
        points=True, verbose=False, keep_just_useful_files=False)
    
    
    # Get transformed landmarks positions
    landmarks = pd.read_csv(lm_out_filepath, header=None, sep='\t |\t', engine='python')
    landmarks.columns = ['point', 'idx', 'input_index', 'input_point', 'ouput_index', 'ouput_point', 'def']
    landmarks_input = [lm[-4:-1] for lm in np.asarray(landmarks.input_index.str.split(' '))]
    landmarks_input = np.asarray(landmarks_input).astype('int')

    landmarks = get_landmarks_array_from_txt_file(lm_out_filepath)
    
    # Get the TRE
    tre = target_registration_error(landmarks_input, sample['e_landmark_pts'], sample['ref_metadata']['spacing'])
    print(f'TRE estimated: {tre}')
    tre = target_registration_error(landmarks, sample['e_landmark_pts'], sample['ref_metadata']['spacing'])
    print(f'TRE estimated: {tre}')
    tre_gt = target_registration_error(sample['i_landmark_pts'], sample['e_landmark_pts'], sample['ref_metadata']['spacing'])
    print(f'Initial displacement: {tre_gt}')
    print(f'Initial displacement GT: {sample["disp_mean"]}{sample["disp_mean"]}')
    tres.append(tre)

print(tres)

  0%|          | 0/4 [00:00<?, ?it/s]

Estimating transformation...
Reg time: 51.62588381767273
Transforming points...


 25%|██▌       | 1/4 [01:09<03:28, 69.44s/it]

TRE estimated: (26.33, 11.42)
TRE estimated: (48.75, 20.33)
Initial displacement: (26.33, 11.42)
Initial displacement GT: 25.925.9
Estimating transformation...
Reg time: 49.29538106918335
Transforming points...


 50%|█████     | 2/4 [02:13<02:12, 66.21s/it]

TRE estimated: (21.79, 6.46)
TRE estimated: (33.37, 7.96)
Initial displacement: (21.79, 6.46)
Initial displacement GT: 21.7721.77
Estimating transformation...
Reg time: 55.33826398849487
Transforming points...


 75%|███████▌  | 3/4 [03:27<01:09, 69.64s/it]

TRE estimated: (12.64, 6.38)
TRE estimated: (24.37, 12.94)
Initial displacement: (12.64, 6.38)
Initial displacement GT: 12.2912.29
Estimating transformation...
Reg time: 49.751301288604736
Transforming points...


100%|██████████| 4/4 [04:34<00:00, 68.63s/it]

TRE estimated: (29.58, 12.92)
TRE estimated: (62.32, 28.63)
Initial displacement: (29.58, 12.92)
Initial displacement GT: 30.930.9
[(48.75, 20.33), (33.37, 7.96), (24.37, 12.94), (62.32, 28.63)]


In [ ]:
-f /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/data/dir_lab_copd/copd1/copd1_eBHCT_lm.nii.gz -m /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/data/dir_lab_copd/copd1/copd1_iBHCT_lm.nii.gz -fMask /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/experiments/elastix/Parameters.Par0008.elastic/copd1/res_tmp/e_body_mask_img.nii.g -mMask /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/experiments/elastix/Parameters.Par0008.elastic/copd1/res_tmp/i_body_mask_img.nii.g -out /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/experiments/elastix/Parameters.Par0008.elastic/copd1/res_tmp_/ -p /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/elastix/parameter_maps/Par0008/Parameters.Par0008.affine.txt -p /home/jseia/Desktop/MAIA/classes/spain/mira/final_project/mira_final_project/elastix/parameter_maps/Par0008/Parameters.Par0008.elastic.txt,

In [5]:
# Par 0007:
    # No masks
    
    # Body masks
    [(48.94, 21.16), (36.44, 10.37), (22.75, 13.28), (57.97, 28.67)]
    
    # Lung masks

    # Coarse:
    [(48.72, 21.15), (35.26, 10.27), (23.02, 13.39), (57.4, 27.62)]


# Par0003:
    # FG
        # R8
            # body mask

            # lung mask

            # no mask

        # R6
            # body mask

            # lung mask

            # no mask

        # R4
            # body mask

            # lung mask

            # no mask

        # R2
            # body mask
                [(47.39, 17.79), (28.49, 7.52), (23.91, 12.89), (61.77, 28.07)]
            # lung mask

            # no mask
        # R1:
            [(47.0, 17.27), (28.19, 7.58), (23.59, 12.8), (61.29, 27.85)]   
        # Affine:
            [(46.87, 16.64), (28.0, 7.32), (23.05, 12.58), (60.96, 27.78)]
            
            
        [(48.75, 20.33), (33.37, 7.96), (24.37, 12.94), (62.32, 28.63)]


IndentationError: unexpected indent (1051239367.py, line 5)